In [1]:
import pandas as pd
from itertools import product

pd.options.display.max_columns = 100


def generate_test_vectors(n_inputs:int=0,labels:list[str]=[]):
    # labels[0] is MSB label
    if n_inputs:
        return pd.DataFrame(
            [*product(*((False, True) for _ in range(n_inputs)))],
            columns=[chr(65 + _) for _ in reversed(range(n_inputs))],
        )
    return pd.DataFrame(
        [*product(*((False, True) for _ in range(len(labels))))],
        columns=labels,
    )

def calc_cases(df: pd.DataFrame, checks: dict[str, pd.Series]):
    out = pd.DataFrame(index=df.index)
    for var, check in checks.items():
        tmp = df[var].where(check)
        out[f"{var}-0"] = tmp.where(tmp == True)
        out[f"{var}-1"] = tmp.where(tmp == False)
    out[out.notna()] = 1
    return out

# to select test cases:
# - select column with most ones and add it to test vector,
# - repeat, but skip rows, where testing is covered
# - if testing vestor is full, stop

In [2]:
# C17 circuit

df = generate_test_vectors(5)

# define/calculate intermediate signals in the structure
df["N1"] = ~(df.A & df.C)
df["N2"] = ~(df.D & df.C)
df["N3"] = ~(df.N2 & df.B)
df["N4"] = ~(df.N2 & df.E)

# define/calculate outputs
df["Y1"] = ~(df.N1 & df.N3)
df["Y2"] = ~(df.N3 & df.N4)

# define/calculate passthrough masks for signals
checks = {
    "A": (df.C & df.N3),
    "B": (df.N2 & (df.N1 | df.N4)),
    "C": (df.A & df.N3)
    | (df.D & df.B & (df.N1 | df.N4 | df.E))
    | (df.D & df.E & df.N3),
    "D": (df.C & df.B & (df.N1 | df.N4 | df.E)) | (df.C & df.E & df.N3),
    "E": (df.N2 & df.N3),
    "N1": df.N3,
    "N2": (df.D & (df.N1 | df.N4)) | (df.E & df.N3),
    "N3": df.N1 | df.N4,
    "N4": df.N3,
    "Y1": df.index >= 0,
    "Y2": df.index >= 0,
}

# calculate test vectors coverage for a SaF model
out = calc_cases(df, checks)
out.fillna(0, inplace=True)

In [3]:
out.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A-0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
A-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
B-0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0
B-1,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0
C-0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
C-1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0
D-0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
D-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0
E-0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0
E-1,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# NO42 circuit

df = generate_test_vectors(4)

df["N3"] = ~(df.A | df.B)
df["N1"] = ~(df.D | df.C)
df["N2"] = ~(df.N1 & df.N3)

df["Q"] = ~(df.N2)

checks = {
    "A": (~df.B & df.N1),
    "B": (~df.A & df.N1),
    "C": (~df.D & df.N3),
    "D": (~df.C & df.N3),
    "N1": df.N3,
    "N2": df.index >= 0,
    "N3": df.N1,
    "Q": df.index >= 0,
}

out = calc_cases(df, checks)
out.fillna(0, inplace=True)

In [5]:
out.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
A-0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
B-0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
B-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C-0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
C-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
D-0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
D-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
N1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
N1-1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0
